In [1]:
import pandas as pd
from zeep import Client
import hashlib
import os
import pandas as pd
import time

# PUT IN EMAIL/PASSWORD HERE (must register for access):
## I've left in my own for now, but replace with the team one for ease
email = "mrunali@berkeley.edu"
pass_str = "brendaiscool"


wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256(pass_str.encode("utf-8")).hexdigest()
client = Client(wsdl)

In [2]:
rxns = pd.read_csv("./iGem/Data_CSVs/brenda_rxns_incomplete.csv")
rxns.head()

,Unnamed: 0,literatureProducts,literatureSubstrates,organismSubstrates,commentarySubstrates,reversibility,organismProducts,products,commentaryProducts,substrates,ecNumber,pubmedId
0,0,NaN,695811.0,Thermus sp.,1% activity compared to cyclohexanone,?,NaN,? + NAD+,NaN,(-)-carvone + NADH + H+,1.1.1.1,18704396.0
1,1,NaN,695811.0,Thermus sp.,125% activity compared to cyclohexanol,r,NaN,(rac)-3-methylcyclohexanone + NADH + H+,163% activity compared to cyclohexanone,"(1S,3S)-3-methylcyclohexanol + NAD+",1.1.1.1,18704396.0
2,2,NaN,735498.0,Yokenella sp.,"41.5% of the activity with (2E)-but-2-enal, yi...",?,NaN,(2E)-2-methylpent-2-en-1-ol + NADP+,NaN,(2E)-2-methylpent-2-enal + NADPH + H+,1.1.1.1,24509923.0
3,3,NaN,735498.0,Yokenella sp.,"37.4% of the activity with (2E)-but-2-enal, yi...",?,NaN,"(2E)-3,7-dimethylocta-2,6-dien-1-ol + NADP+",NaN,"(2E)-3,7-dimethylocta-2,6-dienal + NADPH + H+",1.1.1.1,24509923.0
4,4,NaN,691867.0,Homo sapiens,substrate of isozyme ADH4,r,NaN,"(2E)-non-2-ene-1,4-diol + NAD+",NaN,(2E)-4-hydroxynon-2-enal + NADH + H+,1.1.1.1,18505683.0


In [3]:
print("{} unique PMIDs and {} total rows".format(len(rxns["pubmedId"].unique()), len(rxns)))


103948 unique PMIDs and 459935 total rows


In [4]:
print("{} 0.0 PMID values which is {}% of the total".format(len(rxns[rxns["pubmedId"] == 0]), 100*len(rxns[rxns["pubmedId"] == 0])/len(rxns)))
#

59847 0.0 PMID values which is 13.012056051398567% of the total


In [5]:
PMrxns = rxns[rxns["pubmedId"] != 0].drop("Unnamed: 0", axis = 1)
PMrxns.head()

,literatureProducts,literatureSubstrates,organismSubstrates,commentarySubstrates,reversibility,organismProducts,products,commentaryProducts,substrates,ecNumber,pubmedId
0,NaN,695811.0,Thermus sp.,1% activity compared to cyclohexanone,?,NaN,? + NAD+,NaN,(-)-carvone + NADH + H+,1.1.1.1,18704396.0
1,NaN,695811.0,Thermus sp.,125% activity compared to cyclohexanol,r,NaN,(rac)-3-methylcyclohexanone + NADH + H+,163% activity compared to cyclohexanone,"(1S,3S)-3-methylcyclohexanol + NAD+",1.1.1.1,18704396.0
2,NaN,735498.0,Yokenella sp.,"41.5% of the activity with (2E)-but-2-enal, yi...",?,NaN,(2E)-2-methylpent-2-en-1-ol + NADP+,NaN,(2E)-2-methylpent-2-enal + NADPH + H+,1.1.1.1,24509923.0
3,NaN,735498.0,Yokenella sp.,"37.4% of the activity with (2E)-but-2-enal, yi...",?,NaN,"(2E)-3,7-dimethylocta-2,6-dien-1-ol + NADP+",NaN,"(2E)-3,7-dimethylocta-2,6-dienal + NADPH + H+",1.1.1.1,24509923.0
4,NaN,691867.0,Homo sapiens,substrate of isozyme ADH4,r,NaN,"(2E)-non-2-ene-1,4-diol + NAD+",NaN,(2E)-4-hydroxynon-2-enal + NADH + H+,1.1.1.1,18505683.0


For PM reactions, we can download from SciHub using PMID. For non-PubMed references, we need to get remainder of reference information to get the DOI using CrossRef so we can download the the papers from SciHub. 

In [6]:
nonPMrxns = rxns[rxns["pubmedId"] == 0].drop("Unnamed: 0", axis = 1).drop("pubmedId", axis = 1)
nonPMrxns.head()

,literatureProducts,literatureSubstrates,organismSubstrates,commentarySubstrates,reversibility,organismProducts,products,commentaryProducts,substrates,ecNumber
10,NaN,659773.0,Neurospora crassa,NaN,NaN,NaN,"(4S,6S)-5,6-dihydro-4-hydroxy-6-methyl-4H-thie...",NaN,"(6S)-5,6-dihydro-6-methyl-4H-thieno[2,3b]thiop...",1.1.1.1
12,NaN,285628.0,Camellia sinensis,43.2% of the activity with ethanol,?,NaN,(E)-hex-2-en-1-one + NADH,NaN,(E)-hex-2-en-1-ol + NAD+,1.1.1.1
19,NaN,688442.0,Devosia riboflavina,55% of activity with N-benzyl-3-pyrrolidinol,?,NaN,2-butanone + NADH + H+,NaN,(R)-2-butanol + NAD+,1.1.1.1
20,NaN,688442.0,Devosia riboflavina KNK10702,55% of activity with N-benzyl-3-pyrrolidinol,?,NaN,2-butanone + NADH + H+,NaN,(R)-2-butanol + NAD+,1.1.1.1
73,NaN,285628.0,Camellia sinensis,9.7% of the activity with ethanol,?,NaN,(Z)-hex-2-en-1-one + NADH,NaN,(Z)-hex-2-en-1-ol + NAD+,1.1.1.1


In [7]:
len(nonPMrxns["literatureSubstrates"].unique())

12064

In [8]:
len(nonPMrxns["literatureSubstrates"])

59847

In [38]:
# use getReferenceById to fill in the missing rows if possible
# no need to run again, we created a reference dictionary as a pickle file that can be imported
# Takes ~30 minutes to run (1640/60 = 27 minutes)

# nonPMlitIDs = nonPMrxns.literatureSubstrates.unique() 
# count1 = 0 
# t0 = time.time()
# refDict = {}
# connectionErrorsIds = {}

# for li in nonPMlitIDs:
#     count1 = count1 + 1
#     if (count1 % 500) == 0:
#         print(str(count1) + " out of " + str(len(nonPMlitIDs)))
#         print(time.time() - t0)
#         print('\n')
#     try:
#         parameters = (email,password,li)
#         data = client.service.getReferenceById(*parameters) 
#         refDict[li] = data
#     except Exception as e:
#         connectionErrorsIds[li] = e
#         continue

# print(time.time() - t0)

# refDict

500 out of 12064
68.64833402633667


1000 out of 12064
137.67523980140686


1500 out of 12064
205.60005593299866


2000 out of 12064
273.31452894210815


2500 out of 12064
340.95826292037964


3000 out of 12064
409.97050285339355


3500 out of 12064
477.7613492012024


4000 out of 12064
545.6175570487976


4500 out of 12064
615.1371879577637


5000 out of 12064
683.222021818161


5500 out of 12064
750.9768698215485


6000 out of 12064
818.2482719421387


6500 out of 12064
885.2385869026184


7000 out of 12064
951.4434330463409


7500 out of 12064
1019.2311279773712


8000 out of 12064
1087.1316289901733


8500 out of 12064
1157.129299879074


9000 out of 12064
1225.2485868930817


9500 out of 12064
1292.8442869186401


10000 out of 12064
1360.9422359466553


10500 out of 12064
1428.3671669960022


11000 out of 12064
1495.6548647880554


11500 out of 12064
1563.5901849269867


12000 out of 12064
1631.1365740299225


1640.9776570796967


{659773.0: {
     'authors': 'Blacklock, T.J.; Sohar, P.; Butcher, J.W.; Lamanec, T.; Grabowski, E.J.J.',
     'title': 'An enantioselective synthesis of the topically-active carbonic anhydrase inhibitor MK-0507:5,6-dihydro-(s)-4-(ethylamino)-(s)-6-mehtyl-4H-thieno[2,3-beta]thiopyran-2-sulfonamide 7,7-dioxide hydrochloride',
     'journal': 'J. Org. Chem.',
     'volume': '58',
     'pages': '1672-1679',
     'year': '1993',
     'pubmedId': '0',
     'textmining': '0'
 }, 285628.0: {
     'authors': 'Hatanaka, A.; Harada, T.',
     'title': 'Purification and properties of alcohol dehydrogenase from tea seeds',
     'journal': 'Agric. Biol. Chem.',
     'volume': '36',
     'pages': '2033-2035',
     'year': '1972',
     'pubmedId': '0',
     'textmining': '0'
 }, 688442.0: {
     'authors': 'Kizaki, N.; Yasohara, Y.; Nagashima, N.; Hasegawa, J.',
     'title': 'Characterization of novel alcohol dehydrogenase of Devosia riboflavina involved in stereoselective reduction of 3-pyrrolidino

In [9]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

In [15]:
with open('refDict.p', 'rb') as fp:
    refDict = pickle.load(fp)

Use reference dictionary to get information needed for CrossRef to get DOIs as a string. 

In [71]:
import lxml.etree as ET
import urllib
import re

def get_doi(li):
    DOIerrors = {}
    data = refDict[li]
    for i in (["journal", "volume", "year","pages"]) {
        
    }
    journal = "".join(data["journal"].split())
    volume = "".join(data["volume"].split())
    year = "".join(data["year"].split())
    start_page = "".join(data["pages"].split()).split("-")[0]
    first_author_last_name = "".join(data["authors"].split(";")[0].split(",")[0].split())
    journal_title = "%20".join(journal)
    params = (first_author_last_name, journal_title, volume, start_page, year)
    url = "https://doi.crossref.org/openurl?pid=mrunali@berkeley.edu&aulast={}&title={}&volume={}&spage={}&date={}&noredirect=true".format(*params)
    
    data = urllib.request.urlopen(url).read()
#     print(data)
#     root = ET.fromstring(data)
#     matches = root.findall('.//doi')[1]
    data = data.decode("utf-8")

    if (data.find('unresolved') != -1):
        DOIerrors[li] = data
        return
    else:
        try:
            doi = re.findall('(<doi.*>)(.*)<\/doi>', data)[0][1]
            return "https://sci-hub.tw/{}".format(doi)
        except Exception as e:
            DOIerrors[li] = data
            print(li)

In [72]:
doi_map = {}
for li in refDict.keys():
    doi_map[li] = get_doi(li)

nonPMrxns['literatureSubstrates'].map(doi_map)
nonPMrxns

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
len(DOIerrors)

In [ ]:
https://github.com/zaytoun/scihub.py